# Show Training Reaction Improvements
Run this script once before adding training reactions, and once after

In [ ]:
from rmgpy.data.rmg import RMGDatabase
from rmgpy.chemkin import saveChemkinFile, saveSpeciesDictionary
from rmgpy.rmg.model import Species, getFamilyLibraryObject, CoreEdgeReactionModel
from rmgpy import settings
from IPython.display import display
from rmgpy.cantherm.output import prettify
from rmgpy.kinetics.kineticsdata import KineticsData
from rmgpy.data.kinetics.family import TemplateReaction
from rmgpy.data.kinetics.depository import DepositoryReaction
from copy import copy

In [ ]:
database = RMGDatabase()
libraries = ['vinylCPD_H']
database.load(settings['database.directory'], kineticsFamilies='all', reactionLibraries = libraries, kineticsDepositories='all')

## step1: find fam_rxn for each lib_rxn

In [ ]:
reactionDict = {}
for libraryName in libraries:
    kineticLibrary = database.kinetics.libraries[libraryName]
    for index, entry in kineticLibrary.entries.iteritems():
        lib_rxn = entry.item
        display(lib_rxn)
        # Let's make RMG generate this reaction from the families!
        fam_rxn_list = []
        rxt_mol_mutation_num = 1
        pdt_mol_mutation_num = 1
        for reactant in lib_rxn.reactants:
            rxt_mol_mutation_num *= len(reactant.molecule)

        for product in lib_rxn.products:
            pdt_mol_mutation_num *= len(product.molecule)

        for mutation_i in range(rxt_mol_mutation_num):
            rxts_mol = [spc.molecule[mutation_i%(len(spc.molecule))] for spc in lib_rxn.reactants]
            pdts_mol = [spc.molecule[0] for spc in lib_rxn.products]
            fam_rxn_list.extend(database.kinetics.generateReactionsFromFamilies(
                        reactants=rxts_mol, products=pdts_mol))

        if len(fam_rxn_list) == 1:
            fam_rxn = fam_rxn_list[0]       
            lib_reactants = [r for r in lib_rxn.reactants]        
            fam_reactants = [r for r in fam_rxn.reactants]
            for lib_reactant in lib_reactants:
                for fam_reactant in fam_reactants:
                    if lib_reactant.isIsomorphic(fam_reactant):
                        fam_reactants.remove(fam_reactant)
                        break

            lib_products = [r for r in lib_rxn.products]        
            fam_products = [r for r in fam_rxn.products]
            for lib_product in lib_products:
                for fam_product in fam_products:
                    if lib_product.isIsomorphic(fam_product):
                        fam_products.remove(fam_product)
                        break

            forward = not (len(fam_reactants) != 0 or len(fam_products) != 0)
            # find the labeled atoms using family and reactants & products from fam_rxn           
            family_database = database.kinetics.families[fam_rxn.family]
            family_database.addAtomLabelsForReaction(fam_rxn)
            fam_rxn.index = lib_rxn.index
            reactionDict[lib_rxn]=fam_rxn
            for spec in fam_rxn.reactants + fam_rxn.products:
                print spec.molecule[0].toSMILES()
        else:
            print 'There was an issue finding a single reaction family for this reaction. Please investigate.'

## step2: get fam_rxn's kinetics

Check what the RMG estimated kinetics are

In [ ]:
# Pick a temperature to evaluate the kinetics
T = 623.0 

In [ ]:
cem = CoreEdgeReactionModel()
cem.kineticsEstimator = 'rate rules'
cem.verboseComments = True

In [ ]:
for libraryName in libraries:
    kineticLibrary = database.kinetics.libraries[libraryName]
    for index, entry in kineticLibrary.entries.iteritems():
        lib_rxn = entry.item
        try:
            fam_rxn = reactionDict[lib_rxn]
        except:
            print 'There was an issue finding a single reaction family for this reaction in step 1. Skipping.'
            continue
        for idx, spec in enumerate(fam_rxn.reactants):
            spec = Species(label=spec.label, molecule=spec.molecule)
            spec.getThermoData()
            fam_rxn.reactants[idx] = spec
        for idx, spec in enumerate(fam_rxn.products):
            spec = Species(label=spec.label, molecule=spec.molecule)
            spec.getThermoData()
            fam_rxn.products[idx] = spec

        family = getFamilyLibraryObject(fam_rxn.family)

        # Set the reaction kinetics
        kinetics, source, entryFamily, isForward = cem.generateKinetics(fam_rxn)
        fam_rxn.kinetics = kinetics
        # Flip the reaction direction if the kinetics are defined in the reverse direction
        if not isForward:
            fam_rxn.reactants, fam_rxn.products = fam_rxn.products, fam_rxn.reactants
            fam_rxn.pairs = [(p,r) for r,p in fam_rxn.pairs]
        if family.ownReverse and hasattr(fam_rxn,'reverse'):
            if not isForward:
                fam_rxn.template = fam_rxn.reverse.template
            # We're done with the "reverse" attribute, so delete it to save a bit of memory
            delattr(fam_rxn,'reverse')

        # convert KineticsData to Arrhenius forms
        if isinstance(fam_rxn.kinetics, KineticsData):
            fam_rxn.kinetics = fam_rxn.kinetics.toArrhenius()
        #  correct barrier heights of estimated kinetics
        if isinstance(fam_rxn,TemplateReaction) or isinstance(fam_rxn,DepositoryReaction): # i.e. not LibraryReaction
            fam_rxn.fixBarrierHeight() # also converts ArrheniusEP to Arrhenius.

        if cem.pressureDependence and fam_rxn.isUnimolecular():
            # If this is going to be run through pressure dependence code,
            # we need to make sure the barrier is positive.
            fam_rxn.fixBarrierHeight(forcePositive=True)
        print '==============='
        print index
        display(lib_rxn)
        print ''
        print 'Library Kinetics'
        print prettify(repr(entry.data))
        
        print ''
        print 'Reaction Family Kinetics After Training'
        print 'Family: {}'.format(fam_rxn.family)
        print prettify(repr(fam_rxn.kinetics))
        
        print ''
        print 'Kinetics evaluated at {} K'.format(T)
        print 'Library Kinetics: {:.2E}'.format(entry.data.getRateCoefficient(T))
        print 'Reaction Family Kinetics After Training: {:.2E}'.format(fam_rxn.kinetics.getRateCoefficient(T))